In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embedding_provider = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# Load and chunk documents
loader = WebBaseLoader(
    ["https://en.wikipedia.org/wiki/President_of_the_United_States"]
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

# Vector store
vector_store = FAISS.from_documents(document_chunks, embedding_provider)
retriever = vector_store.as_retriever(search_type="similarity")

# Contextualize question chain
contextualize_q_system_prompt = """Given a chat history and the latest user question,
which might reference context in the chat history, formulate a standalone question,
which can be understood without the chat history.
Do not answer the question, just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import AIMessage, HumanMessage

qa_system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say you don't know.
Use three sentences maximum and keep it concise.

{context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain.invoke(input)
    return input["question"]

rag_chain = (
    RunnablePassthrough.assign(
        context=RunnableLambda(contextualized_question) | retriever | format_docs
    )
    | qa_prompt
    | llm
    | StrOutputParser()  # Extract string from AIMessage
)

# Usage
chat_history = []
question = "What is a large language model?"

response = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(response)

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response),  # Now `response` is a string
])

A large language model is an artificial intelligence system designed to understand and generate human language. It is trained on vast amounts of text data to learn patterns, grammar, and context, enabling it to produce coherent and contextually relevant responses. These models are used in various applications, including chatbots, translation services, and content generation.


In [6]:
second_question = "Who is the current president of the United States?"
second_answer = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
print(second_answer)

The current president of the United States is Donald Trump, who has been in office since 2025.
